In [123]:
import re 
import time
import requests
import numpy as np
import pandas as pd
from scrapy.http import TextResponse

from textblob import TextBlob
import nltk
from nltk.corpus import stopwords , wordnet
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [38]:
from pprint import pprint

In [2]:
url  = "https://www.rogerebert.com/reviews"

In [4]:
page = requests.get(url)
response = TextResponse(url=page.url,body = page.text,encoding='utf-8')

In [6]:
response.css("h5.title> a::text").extract_first()

'Skin'

In [7]:
response.css("figure[class= 'movie review']> p.byline::text").extract_first()

'\nChristy Lemire\n'

In [10]:
response.css("span[class= 'release-year']::text").re("\d{4}")
# or response.css("span.release-year::text").extract_first()[1,-1]

['2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019']

In [17]:
response.css("span.star-rating").extract()[0].count("full") + 0.5*response.css("span.star-rating").extract()[0].count("half")

4.0

In [22]:
"https://www.rogerebert.com" + response.css("h5.title > a::attr(href)").extract_first()

'https://www.rogerebert.com/reviews/skin-2019'

In [59]:
def review_info_scraper(url):
    page = requests.get(url)
    response = TextResponse(url=page.url,body = page.text,encoding='utf-8')
    try:
        title = response.css("h5.title> a::text").extract()
        author = response.css("figure[class= 'movie review']> p.byline::text").extract()
        year = response.css("span[class= 'release-year']::text").re("\d{4}")
        rating =[i.count("full")+0.5* i.count("half") for i in response.css("span.star-rating").extract()]
        link = ["https://www.rogerebert.com" + i for i in  response.css("h5.title > a::attr(href)").extract()]
        movie_info ={"title":title, "author":author, "year":year, "rating":rating,"link":link}
    except:
        error.append(url)
    return movie_info

In [41]:
movie_data = review_info_scraper(url)

In [42]:
pprint(movie_data)

{'author': ['\nChristy Lemire\n',
            "\nSheila O'Malley\n",
            '\nGlenn Kenny\n',
            '\nMonica Castillo\n',
            '\nMatt Fagerholm\n',
            '\nTomris Laffly\n',
            '\nOdie Henderson\n',
            '\nBrian Tallerico\n',
            '\nBrian Tallerico\n',
            '\nChristy Lemire\n',
            '\nMatt Zoller Seitz\n',
            '\nBrian Tallerico\n',
            '\nGlenn Kenny\n',
            '\nPeter Sobczynski\n',
            '\nBrian Tallerico\n',
            '\nScout Tafoya\n',
            '\nSimon Abrams\n',
            '\nNell Minow \n',
            '\nMatt Fagerholm\n',
            '\nGodfrey Cheshire\n',
            '\nMatt Zoller Seitz\n',
            '\nMatt Zoller Seitz\n',
            '\nTomris Laffly\n',
            '\nChristy Lemire\n'],
 'link': ['https://www.rogerebert.com/reviews/skin-2019',
          'https://www.rogerebert.com/reviews/the-mountain-2019',
          'https://www.rogerebert.com/reviews/mike-wall

In [25]:
[i.count("full")+0.5* i.count("half") for i in response.css("span.star-rating").extract()]

[4.0,
 3.0,
 3.0,
 3.0,
 3.5,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 3.0,
 3.0,
 3.5,
 4.0,
 4.0,
 3.0,
 4.0,
 3.5,
 2.5,
 3.0,
 2.5,
 3.5,
 2.0,
 2.5,
 3.5,
 0.5,
 3.0,
 3.5,
 3.5,
 2.5,
 2.5,
 2.5,
 4.0]

In [30]:
["https://www.rogerebert.com" + i for i in response.css("h5.title> a::attr(href)").extract()]

['https://www.rogerebert.com/reviews/skin-2019',
 'https://www.rogerebert.com/reviews/the-mountain-2019',
 'https://www.rogerebert.com/reviews/mike-wallace-is-here-2019',
 'https://www.rogerebert.com/reviews/honeyland-2019',
 'https://www.rogerebert.com/reviews/share-2019',
 'https://www.rogerebert.com/reviews/for-sama-2019',
 'https://www.rogerebert.com/reviews/the-great-hack-2019',
 'https://www.rogerebert.com/reviews/once-upon-a-time-in--hollywood-2019',
 'https://www.rogerebert.com/reviews/who-killed-garrett-phillips-2019',
 'https://www.rogerebert.com/reviews/this-changes-everything-2019',
 'https://www.rogerebert.com/reviews/the-lion-king-2019',
 'https://www.rogerebert.com/reviews/luz-2019',
 'https://www.rogerebert.com/reviews/a-faithful-man-2019',
 'https://www.rogerebert.com/reviews/into-the-ashes-2019',
 'https://www.rogerebert.com/reviews/cooked-survival-by-zip-code',
 'https://www.rogerebert.com/reviews/coincoin-and-the-extra-humans-2019',
 'https://www.rogerebert.com/revi

In [47]:
page = requests.get("https://www.rogerebert.com/reviews/skin-2019")


In [50]:
response = TextResponse(url=page.url,body = page.text,encoding='utf-8')

In [51]:
response.css("div[itemprop='reviewBody'] > p ::text").extract()

['A middle-aged white man stands on an outdoor stage, addressing an enthusiastic audience, whipping his followers into a frenzy. He says he plans to run for the Ohio state legislature, and his disparagement of Asians, blacks and other minorities (although he uses far more offensive language to describe them) is the central premise of his platform.',
 '“Maybe we should just make them leave,” he suggests forcefully, and he’s greeted with wild cheers of agreement in response.',
 'It’s a chilling echo of the kind of “Send her back” chants we’ve heard at a recent presidential rally and seen on Twitter. But it takes place during an early scene in “',
 'Skin',
 ',” and it’s set more than a decade ago. The drama, inspired by the true story of a young man who dared to escape his white supremacist group and remove his many racist tattoos, sadly couldn’t be more relevant. Bryon Widner fled the violence and hate that defined his life for so long, but they aren’t going anywhere.',
 'Writer/director

In [57]:
str_ = " ".join(response.css("div[itemprop='reviewBody'] > p ::text").extract())

In [64]:
dictionary = review_info_scraper("https://www.rogerebert.com/reviews")

In [67]:
def get_desc(url):
    page = requests.get(url)
    response = TextResponse(url=page.url,body = page.text,encoding='utf-8')
    desc_list=response.css("div[itemprop='reviewBody'] > p ::text").extract()
    desc = " ".join(desc_list)
    return desc

In [68]:
all_description = []
for i in dictionary ["link"]:
    all_description.append(get_desc(i))
    time.sleep(2)

In [84]:
join = ' '.join(all_description).lower()

In [72]:
# clean textual data 
# make lowercase , remove stopwords , remove punct , lem., stem,

In [74]:
sw = stopwords.words('english')

In [86]:
word = TextBlob(join)

In [87]:
word.words

WordList(['a', 'middle-aged', 'white', 'man', 'stands', 'on', 'an', 'outdoor', 'stage', 'addressing', 'an', 'enthusiastic', 'audience', 'whipping', 'his', 'followers', 'into', 'a', 'frenzy', 'he', 'says', 'he', 'plans', 'to', 'run', 'for', 'the', 'ohio', 'state', 'legislature', 'and', 'his', 'disparagement', 'of', 'asians', 'blacks', 'and', 'other', 'minorities', 'although', 'he', 'uses', 'far', 'more', 'offensive', 'language', 'to', 'describe', 'them', 'is', 'the', 'central', 'premise', 'of', 'his', 'platform', '“', 'maybe', 'we', 'should', 'just', 'make', 'them', 'leave', '”', 'he', 'suggests', 'forcefully', 'and', 'he', '’', 's', 'greeted', 'with', 'wild', 'cheers', 'of', 'agreement', 'in', 'response', 'it', '’', 's', 'a', 'chilling', 'echo', 'of', 'the', 'kind', 'of', '“', 'send', 'her', 'back', '”', 'chants', 'we', '’', 've', 'heard', 'at', 'a', 'recent', 'presidential', 'rally', 'and', 'seen', 'on', 'twitter', 'but', 'it', 'takes', 'place', 'during', 'an', 'early', 'scene', 'in',

In [91]:
# isalnum() supposed to eliminate '' s

In [90]:
word_sw = [i for i in word.words.lower() if i not in sw and i.isalnum()]

In [92]:
word_sw

['white',
 'man',
 'stands',
 'outdoor',
 'stage',
 'addressing',
 'enthusiastic',
 'audience',
 'whipping',
 'followers',
 'frenzy',
 'says',
 'plans',
 'run',
 'ohio',
 'state',
 'legislature',
 'disparagement',
 'asians',
 'blacks',
 'minorities',
 'although',
 'uses',
 'far',
 'offensive',
 'language',
 'describe',
 'central',
 'premise',
 'platform',
 'maybe',
 'make',
 'leave',
 'suggests',
 'forcefully',
 'greeted',
 'wild',
 'cheers',
 'agreement',
 'response',
 'chilling',
 'echo',
 'kind',
 'send',
 'back',
 'chants',
 'heard',
 'recent',
 'presidential',
 'rally',
 'seen',
 'twitter',
 'takes',
 'place',
 'early',
 'scene',
 'skin',
 'set',
 'decade',
 'ago',
 'drama',
 'inspired',
 'true',
 'story',
 'young',
 'man',
 'dared',
 'escape',
 'white',
 'supremacist',
 'group',
 'remove',
 'many',
 'racist',
 'tattoos',
 'sadly',
 'relevant',
 'bryon',
 'widner',
 'fled',
 'violence',
 'hate',
 'defined',
 'life',
 'long',
 'going',
 'anywhere',
 'guy',
 'nattiv',
 'film',
 'exa

In [95]:
all_desc = []
for j in all_description: 
    all_words = TextBlob(j).words.lower()
    current_desc =""
    for i in all_words:
        if i not in sw and i.isalnum():
            current_desc = current_desc+i
            all_desc.append(current_desc)
                                

In [96]:
all_desc_clean = [" ".join([i for i in TextBlob(j).words.lower() if i not in sw and i.isalnum() ]) for j in all_description]

In [98]:
all_desc_clean

['white man stands outdoor stage addressing enthusiastic audience whipping followers frenzy says plans run ohio state legislature disparagement asians blacks minorities although uses far offensive language describe central premise platform maybe make leave suggests forcefully greeted wild cheers agreement response chilling echo kind send back chants heard recent presidential rally seen twitter takes place early scene skin set decade ago drama inspired true story young man dared escape white supremacist group remove many racist tattoos sadly relevant bryon widner fled violence hate defined life long going anywhere guy nattiv film exactly version short although happens carry title films depict clashes white supremacist gangs black activists feature patti cake star danielle macdonald crucial role explore unflinching fashion means wear beliefs outwardly brash intimidation factor imagery creates vow loyalty suggests skin focuses widner allegiance vinlanders social club key figure authority 

In [101]:
tf_idf = TfidfVectorizer()

In [102]:
tfidf_matrix = tf_idf.fit_transform(all_desc_clean)

In [103]:
tfidf_matrix

<24x5216 sparse matrix of type '<class 'numpy.float64'>'
	with 9113 stored elements in Compressed Sparse Row format>

In [108]:
words = tf_idf.get_feature_names()
data =tfidf_matrix.toarray()
df = pd.DataFrame(data,columns=words)

In [109]:
df.head()

,100,106,10s,11,15,16,16mm,17,18,1950s,...,zealots,zealous,zeros,zhao,zhou,zimbardo,zingers,zip,zoe,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.051184,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.091871,...,0.000000,0.034603,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.034951,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [111]:
algo = KMeans(3)

In [112]:
algo.fit_transform(df)

array([[1.00277175, 1.21407108, 0.78963274],
       [1.00196096, 0.67348205, 1.13422218],
       [1.01590466, 0.67348205, 1.14893047],
       [0.95705586, 1.21634671, 1.13249198],
       [0.9981703 , 1.2103379 , 0.78771281],
       [0.94139352, 1.20254934, 1.11174498],
       [0.95079267, 1.21193588, 1.13344426],
       [0.9281797 , 1.19414077, 1.11572857],
       [0.95491885, 1.2122644 , 1.13357897],
       [0.93368353, 1.19118101, 1.11275286],
       [0.92184344, 1.18950008, 1.11780939],
       [0.95829447, 1.21455346, 1.138192  ],
       [0.95967644, 1.22070391, 1.14123023],
       [0.93458851, 1.202225  , 1.11864437],
       [0.9489805 , 1.20903381, 1.13713084],
       [0.94520413, 1.20578806, 1.1269432 ],
       [0.95191241, 1.21993542, 1.13908889],
       [0.9490532 , 1.21316376, 1.12682609],
       [0.94855097, 1.21048477, 1.12447628],
       [0.95292845, 1.21366523, 1.12501072],
       [0.96370922, 1.21720989, 1.14736436],
       [0.94602293, 1.21104185, 1.12277745],
       [0.

In [115]:
algo.fit_predict(df)

array([1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1])

In [116]:
df["cluster"] = algo.fit_predict(df)

In [120]:
df["names"] = movie_data["title"]

In [122]:
df[df["cluster"]==1].names

12    A Faithful Man
Name: names, dtype: object